In [ ]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
from tqdm import tqdm

# Configuration du périphérique
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Définition du modèle de débruitage (CGNet)
class CGNet(nn.Module):
    def __init__(self):
        super(CGNet, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1),
        )

    def forward(self, x):
        return self.model(x)

# Charger le modèle préentraîné
generator = CGNet().to(device)
generator.load_state_dict(torch.load("generator_highfreq.pth", map_location=device))
generator.eval()

# Ajouter du bruit combiné
def add_combined_noise(frame, mean=0, std=20, prob_sp=0.05):
    noisy_frame = frame.astype(np.float32)

    # Bruit gaussien
    noise_gaussian = np.random.normal(mean, std, noisy_frame.shape).astype(np.float32)
    noisy_frame = np.clip(noisy_frame + noise_gaussian, 0, 255)

    # Bruit sel et poivre
    salt_pepper = np.random.uniform(0, 1, noisy_frame.shape[:2])
    noisy_frame[salt_pepper < (prob_sp / 2)] = 0  # Sel (noir)
    noisy_frame[salt_pepper > 1 - (prob_sp / 2)] = 255  # Poivre (blanc)

    return noisy_frame.astype(np.uint8)

# Convertir une image numpy en Tensor pour le modèle
def image_to_tensor(image):
    image = torch.from_numpy(image.transpose(2, 0, 1)).float().unsqueeze(0).to(device)  # HWC -> CHW
    return image

# Convertir une Tensor en image numpy
def tensor_to_image(tensor):
    tensor = tensor.squeeze(0).permute(1, 2, 0)  # CHW -> HWC
    image = tensor.cpu().numpy()
    image = np.clip(image, 0, 255).astype(np.uint8)
    return image

# Traitement des frames et création des vidéos
def process_video_frames(video_folder, noised_folder, denoised_folder):
    video_name = os.path.basename(video_folder)
    noised_video_path = os.path.join(noised_folder, f"{video_name}.mp4")
    denoised_video_path = os.path.join(denoised_folder, f"{video_name}.mp4")

    # Liste des frames triées par ordre
    frames = sorted([f for f in os.listdir(video_folder) if f.endswith(('.jpg', '.png'))])

    # Obtenir les dimensions de la première frame
    first_frame_path = os.path.join(video_folder, frames[0])
    first_frame = cv2.imread(first_frame_path)
    height, width, _ = first_frame.shape

    # Initialiser les écrivains de vidéos
    noised_writer = cv2.VideoWriter(noised_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 25, (width, height))
    denoised_writer = cv2.VideoWriter(denoised_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 25, (width, height))

    for frame_name in tqdm(frames, desc=f"Processing video {video_name}"):
        frame_path = os.path.join(video_folder, frame_name)
        frame = cv2.imread(frame_path)

        if frame is None:
            print(f"Impossible de lire {frame_path}")
            continue

        # Ajouter du bruit
        noisy_frame = add_combined_noise(frame)

        # Convertir en tensor
        noisy_frame_tensor = image_to_tensor(noisy_frame)

        # Débruiter avec le modèle
        with torch.no_grad():
            denoised_frame_tensor = generator(noisy_frame_tensor)

        # Convertir en image
        denoised_frame = tensor_to_image(denoised_frame_tensor)

        # Ajouter les frames aux vidéos
        noised_writer.write(noisy_frame)
        denoised_writer.write(denoised_frame)

    # Libérer les ressources
    noised_writer.release()
    denoised_writer.release()
    print(f"Vidéos sauvegardées :\n - Noised : {noised_video_path}\n - Denoised : {denoised_video_path}")

# Fonction principale pour traiter toutes les vidéos
def process_all_videos(input_dir, noised_dir, denoised_dir):
    if not os.path.exists(noised_dir):
        os.makedirs(noised_dir)
    if not os.path.exists(denoised_dir):
        os.makedirs(denoised_dir)

    video_folders = [os.path.join(input_dir, d) for d in os.listdir(input_dir) if os.path.isdir(os.path.join(input_dir, d))]

    for video_folder in video_folders:
        process_video_frames(video_folder, noised_dir, denoised_dir)

# Exemple d'utilisation
input_directory = "DAVIS/DAVIS/JPEGImages/Full-Resolution"  # Chemin vers les vidéos
noised_directory = "video_noised"  # Dossier des vidéos bruitées
denoised_directory = "video_denoised"  # Dossier des vidéos débruitées

process_all_videos(input_directory, noised_directory, denoised_directory)

C:\Users\alari\AppData\Local\Temp\ipykernel_43104\1959180368.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  generator.load_state_dict(torch.load("generator_highfreq.pt

KeyboardInterrupt: 

In [46]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
from tqdm import tqdm

# Configuration du périphérique
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Définition du modèle de débruitage (CGNet)
class CGNet(nn.Module):
    def __init__(self):
        super(CGNet, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True)
        )
        self.middle = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.middle(x)
        x = self.decoder(x)
        return x

# Charger le modèle préentraîné avec chargement strict
generator = CGNet().to(device)
state_dict = torch.load("cgnet_gauss_nima_ssim2.pth", map_location=device)
generator.load_state_dict(state_dict, strict=True)
generator.eval()

# Ajouter du bruit gaussien
def add_gaussian_noise(frame, mean=0, std=25):
    noisy_frame = frame.astype(np.float32)
    noise = np.random.normal(mean, std, noisy_frame.shape).astype(np.float32)
    noisy_frame = np.clip(noisy_frame + noise, 0, 255)
    return noisy_frame.astype(np.uint8)

# Convertir une image numpy en Tensor pour le modèle
def image_to_tensor(image):
    image = torch.from_numpy(image.transpose(2, 0, 1)).float().unsqueeze(0).to(device) / 255.0
    return image

# Convertir une Tensor en image numpy
def tensor_to_image(tensor):
    tensor = tensor.squeeze(0).permute(1, 2, 0)
    image = (tensor.cpu().numpy() * 255).clip(0, 255).astype(np.uint8)
    return image

# Traitement des frames et création des vidéos
def process_video_frames(video_folder, noised_folder, denoised_folder):
    video_name = os.path.basename(video_folder)
    noised_video_path = os.path.join(noised_folder, f"{video_name}.mp4")
    denoised_video_path = os.path.join(denoised_folder, f"{video_name}.mp4")

    # Liste des frames triées par ordre
    frames = sorted([f for f in os.listdir(video_folder) if f.endswith(('.jpg', '.png'))])

    # Obtenir les dimensions de la première frame
    first_frame_path = os.path.join(video_folder, frames[0])
    first_frame = cv2.imread(first_frame_path)
    height, width, _ = first_frame.shape

    # Initialiser les écrivains de vidéos
    noised_writer = cv2.VideoWriter(noised_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 25, (width, height))
    denoised_writer = cv2.VideoWriter(denoised_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 25, (width, height))

    for frame_name in tqdm(frames, desc=f"Processing video {video_name}"):
        frame_path = os.path.join(video_folder, frame_name)
        frame = cv2.imread(frame_path)

        if frame is None:
            print(f"Impossible de lire {frame_path}")
            continue

        # Conversion BGR -> RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Ajouter du bruit gaussien
        noisy_frame = add_gaussian_noise(frame)

        # Convertir en tensor
        noisy_frame_tensor = image_to_tensor(noisy_frame)

        # Débruiter avec le modèle
        with torch.no_grad():
            denoised_frame_tensor = generator(noisy_frame_tensor)

        # Convertir en image
        denoised_frame = tensor_to_image(denoised_frame_tensor)

        # Conversion RGB -> BGR pour OpenCV
        noisy_frame = cv2.cvtColor(noisy_frame, cv2.COLOR_RGB2BGR)
        denoised_frame = cv2.cvtColor(denoised_frame, cv2.COLOR_RGB2BGR)

        # Ajouter les frames aux vidéos
        noised_writer.write(noisy_frame)
        denoised_writer.write(denoised_frame)

    # Libérer les ressources
    noised_writer.release()
    denoised_writer.release()
    print(f"Vidéos sauvegardées :\n - Noised : {noised_video_path}\n - Denoised : {denoised_video_path}")

# Fonction principale pour traiter toutes les vidéos du dataset DAVIS
def process_all_videos(input_dir, noised_dir, denoised_dir):
    if not os.path.exists(noised_dir):
        os.makedirs(noised_dir)
    if not os.path.exists(denoised_dir):
        os.makedirs(denoised_dir)

    video_folders = [os.path.join(input_dir, d) for d in os.listdir(input_dir) if os.path.isdir(os.path.join(input_dir, d))]

    for video_folder in video_folders:
        process_video_frames(video_folder, noised_dir, denoised_dir)

# Exemple d'utilisation sur le dataset DAVIS
input_directory = "DAVIS/DAVIS/JPEGImages/Full-Resolution"  # Chemin vers les vidéos DAVIS
noised_directory = "video_noised_gaussien"  # Dossier pour sauvegarder les vidéos bruitées
denoised_directory = "video_denoised_gaussien"  # Dossier pour sauvegarder les vidéos débruitées

process_all_videos(input_directory, noised_directory, denoised_directory)


C:\Users\alari\AppData\Local\Temp\ipykernel_43104\4215793947.py:42: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("cgnet_gauss_nima_ssim2.pth", map_l

Vidéos sauvegardées :
 - Noised : video_noised_gaussien\bear.mp4
 - Denoised : video_denoised_gaussien\bear.mp4


Processing video bike-packing: 100%|██████████| 69/69 [15:44<00:00, 13.69s/it]


Vidéos sauvegardées :
 - Noised : video_noised_gaussien\bike-packing.mp4
 - Denoised : video_denoised_gaussien\bike-packing.mp4


Processing video blackswan: 100%|██████████| 50/50 [01:17<00:00,  1.54s/it]


...

Vidéos sauvegardées :
 - Noised : video_noised_gaussien\breakdance.mp4
 - Denoised : video_denoised_gaussien\breakdance.mp4


Processing video breakdance-flare: 100%|██████████| 71/71 [01:44<00:00,  1.47s/it]


Vidéos sauvegardées :
 - Noised : video_noised_gaussien\breakdance-flare.mp4
 - Denoised : video_denoised_gaussien\breakdance-flare.mp4


Processing video bus:  29%|██▉       | 23/80 [00:36<01:30,  1.59s/it]


KeyboardInterrupt: 

In [49]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
from tqdm import tqdm

# Configuration du périphérique
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Définition du modèle de débruitage (CGNet)
class CGNet(nn.Module):
    def __init__(self):
        super(CGNet, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True)
        )
        self.middle = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.middle(x)
        x = self.decoder(x)
        return x

# Charger le modèle préentraîné avec chargement strict
generator = CGNet().to(device)
state_dict = torch.load("cgnet_sel_poivre_nima_ssim.pth", map_location=device)
generator.load_state_dict(state_dict, strict=True)  # Chargement strict
generator.eval()

# Ajouter du bruit sel et poivre
def add_salt_and_pepper_noise(frame, prob_sp=0.05):
    noisy_frame = frame.copy()
    salt_pepper = np.random.rand(*noisy_frame.shape[:2])

    # Sel (pixels blancs)
    noisy_frame[salt_pepper < (prob_sp / 2)] = 0
    # Poivre (pixels noirs)
    noisy_frame[salt_pepper > 1 - (prob_sp / 2)] = 255

    return noisy_frame

# Fonction de train
def trait(image, kernel_size=3):
    filtered_image = cv2.medianBlur(image, kernel_size)
    return filtered_image

# Convertir une image numpy en Tensor pour le modèle
def image_to_tensor(image):
    image = torch.from_numpy(image.transpose(2, 0, 1)).float().unsqueeze(0).to(device) / 255.0  # Normalisation
    return image

# Convertir une Tensor en image numpy
def tensor_to_image(tensor):
    tensor = tensor.squeeze(0).permute(1, 2, 0)  # CHW -> HWC
    image = (tensor.cpu().numpy() * 255).clip(0, 255).astype(np.uint8)  # Dénormalisation
    return image

# Traitement des frames et création des vidéos
def process_video_frames(video_folder, noised_folder, denoised_folder):
    video_name = os.path.basename(video_folder)
    noised_video_path = os.path.join(noised_folder, f"{video_name}.mp4")
    denoised_video_path = os.path.join(denoised_folder, f"{video_name}.mp4")

    # Liste des frames triées par ordre
    frames = sorted([f for f in os.listdir(video_folder) if f.endswith(('.jpg', '.png'))])

    # Obtenir les dimensions de la première frame
    first_frame_path = os.path.join(video_folder, frames[0])
    first_frame = cv2.imread(first_frame_path)
    height, width, _ = first_frame.shape

    # Initialiser les écrivains de vidéos
    noised_writer = cv2.VideoWriter(noised_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 25, (width, height))
    denoised_writer = cv2.VideoWriter(denoised_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 25, (width, height))

    for frame_name in tqdm(frames, desc=f"Processing video {video_name}"):
        frame_path = os.path.join(video_folder, frame_name)
        frame = cv2.imread(frame_path)

        if frame is None:
            print(f"Impossible de lire {frame_path}")
            continue

        # Conversion BGR -> RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Ajouter du bruit sel et poivre
        noisy_frame = add_salt_and_pepper_noise(frame)

        # Appliquer le filtre médian avec la fonction trait
        filtered_frame = trait(noisy_frame)

        # Convertir en tensor
        filtered_frame_tensor = image_to_tensor(filtered_frame)

        # Débruiter avec le modèle
        with torch.no_grad():
            denoised_frame_tensor = generator(filtered_frame_tensor)

        # Convertir en image
        denoised_frame = tensor_to_image(denoised_frame_tensor)

        # Conversion RGB -> BGR pour OpenCV
        noisy_frame = cv2.cvtColor(noisy_frame, cv2.COLOR_RGB2BGR)
        denoised_frame = cv2.cvtColor(denoised_frame, cv2.COLOR_RGB2BGR)

        # Ajouter les frames aux vidéos
        noised_writer.write(noisy_frame)
        denoised_writer.write(denoised_frame)

    # Libérer les ressources
    noised_writer.release()
    denoised_writer.release()
    print(f"Vidéos sauvegardées :\n - Noised : {noised_video_path}\n - Denoised : {denoised_video_path}")

# Fonction principale pour traiter toutes les vidéos du dataset DAVIS
def process_all_videos(input_dir, noised_dir, denoised_dir):
    if not os.path.exists(noised_dir):
        os.makedirs(noised_dir)
    if not os.path.exists(denoised_dir):
        os.makedirs(denoised_dir)

    video_folders = [os.path.join(input_dir, d) for d in os.listdir(input_dir) if os.path.isdir(os.path.join(input_dir, d))]

    for video_folder in video_folders:
        process_video_frames(video_folder, noised_dir, denoised_dir)

# Exemple d'utilisation sur le dataset DAVIS
input_directory = "DAVIS/DAVIS/JPEGImages/Full-Resolution"  # Chemin vers les vidéos DAVIS
noised_directory = "video_noised_s&p"  # Dossier pour sauvegarder les vidéos bruitées
denoised_directory = "video_denoised_s&p"  # Dossier pour sauvegarder les vidéos débruitées

process_all_videos(input_directory, noised_directory, denoised_directory)


C:\Users\alari\AppData\Local\Temp\ipykernel_43104\3613855664.py:42: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("cgnet_sel_poivre_nima_ssim.pth", m

Vidéos sauvegardées :
 - Noised : video_noised_s&p\bear.mp4
 - Denoised : video_denoised_s&p\bear.mp4


Processing video bike-packing: 100%|██████████| 69/69 [09:40<00:00,  8.41s/it]


Vidéos sauvegardées :
 - Noised : video_noised_s&p\bike-packing.mp4
 - Denoised : video_denoised_s&p\bike-packing.mp4


Processing video blackswan: 100%|██████████| 50/50 [01:13<00:00,  1.47s/it]


Vidéos sauvegardées :
 - Noised : video_noised_s&p\blackswan.mp4
 - Denoised : video_denoised_s&p\blackswan.mp4


Processing video bmx-bumps:  11%|█         | 10/90 [00:15<02:01,  1.52s/it]


KeyboardInterrupt: 

In [50]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
from tqdm import tqdm

# Configuration du périphérique
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Définition du modèle de débruitage (CGNet)
class CGNet(nn.Module):
    def __init__(self):
        super(CGNet, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True)
        )
        self.middle = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.middle(x)
        x = self.decoder(x)
        return x

# Charger le modèle préentrainé avec chargement strict
generator = CGNet().to(device)
state_dict = torch.load("cgnet_combinaison_nima_ssim.pth", map_location=device)
generator.load_state_dict(state_dict, strict=True)
generator.eval()

# Ajouter une combinaison de bruit gaussien et sel et poivre
def add_combined_noise(frame, mean=0, std=25, prob_sp=0.05):
    # Bruit gaussien
    noisy_frame = frame.astype(np.float32)
    noise = np.random.normal(mean, std, noisy_frame.shape).astype(np.float32)
    noisy_frame = np.clip(noisy_frame + noise, 0, 255)

    # Bruit sel et poivre
    salt_pepper = np.random.rand(*noisy_frame.shape[:2])
    noisy_frame[salt_pepper < (prob_sp / 2)] = 0       # Sel (noir)
    noisy_frame[salt_pepper > 1 - (prob_sp / 2)] = 255 # Poivre (blanc)
    
    return noisy_frame.astype(np.uint8)

# Convertir une image numpy en Tensor pour le modèle
def image_to_tensor(image):
    image = torch.from_numpy(image.transpose(2, 0, 1)).float().unsqueeze(0).to(device) / 255.0
    return image

# Convertir une Tensor en image numpy
def tensor_to_image(tensor):
    tensor = tensor.squeeze(0).permute(1, 2, 0)
    image = (tensor.cpu().numpy() * 255).clip(0, 255).astype(np.uint8)
    return image

# Traitement des frames et création des vidéos
def process_video_frames(video_folder, noised_folder, denoised_folder):
    video_name = os.path.basename(video_folder)
    noised_video_path = os.path.join(noised_folder, f"{video_name}.mp4")
    denoised_video_path = os.path.join(denoised_folder, f"{video_name}.mp4")

    # Liste des frames triées par ordre
    frames = sorted([f for f in os.listdir(video_folder) if f.endswith(('.jpg', '.png'))])

    # Obtenir les dimensions de la première frame
    first_frame_path = os.path.join(video_folder, frames[0])
    first_frame = cv2.imread(first_frame_path)
    height, width, _ = first_frame.shape

    # Initialiser les écrivains de vidéos
    noised_writer = cv2.VideoWriter(noised_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 25, (width, height))
    denoised_writer = cv2.VideoWriter(denoised_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 25, (width, height))

    for frame_name in tqdm(frames, desc=f"Processing video {video_name}"):
        frame_path = os.path.join(video_folder, frame_name)
        frame = cv2.imread(frame_path)

        if frame is None:
            print(f"Impossible de lire {frame_path}")
            continue

        # Conversion BGR -> RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Ajouter la combinaison de bruit gaussien et sel et poivre
        noisy_frame = add_combined_noise(frame)

        # Convertir en tensor
        noisy_frame_tensor = image_to_tensor(noisy_frame)

        # Débruiter avec le modèle
        with torch.no_grad():
            denoised_frame_tensor = generator(noisy_frame_tensor)

        # Convertir en image
        denoised_frame = tensor_to_image(denoised_frame_tensor)

        # Conversion RGB -> BGR pour OpenCV
        noisy_frame = cv2.cvtColor(noisy_frame, cv2.COLOR_RGB2BGR)
        denoised_frame = cv2.cvtColor(denoised_frame, cv2.COLOR_RGB2BGR)

        # Ajouter les frames aux vidéos
        noised_writer.write(noisy_frame)
        denoised_writer.write(denoised_frame)

    # Libérer les ressources
    noised_writer.release()
    denoised_writer.release()
    print(f"Vidéos sauvegardées :\n - Noised : {noised_video_path}\n - Denoised : {denoised_video_path}")

# Fonction principale pour traiter toutes les vidéos du dataset DAVIS
def process_all_videos(input_dir, noised_dir, denoised_dir):
    if not os.path.exists(noised_dir):
        os.makedirs(noised_dir)
    if not os.path.exists(denoised_dir):
        os.makedirs(denoised_dir)

    video_folders = [os.path.join(input_dir, d) for d in os.listdir(input_dir) if os.path.isdir(os.path.join(input_dir, d))]

    for video_folder in video_folders:
        process_video_frames(video_folder, noised_dir, denoised_dir)

# Exemple d'utilisation sur le dataset DAVIS
input_directory = "DAVIS/DAVIS/JPEGImages/Full-Resolution"  # Chemin vers les vidéos DAVIS
noised_directory = "video_noised_combi"  # Dossier pour sauvegarder les vidéos bruitées
denoised_directory = "video_denoised_combi"  # Dossier pour sauvegarder les vidéos débruitées

process_all_videos(input_directory, noised_directory, denoised_directory)


C:\Users\alari\AppData\Local\Temp\ipykernel_43104\1415137233.py:42: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("cgnet_combinaison_nima_ssim.pth", 

Vidéos sauvegardées :
 - Noised : video_noised_combi\bear.mp4
 - Denoised : video_denoised_combi\bear.mp4


Processing video bike-packing: 100%|██████████| 69/69 [11:09<00:00,  9.70s/it]


Vidéos sauvegardées :
 - Noised : video_noised_combi\bike-packing.mp4
 - Denoised : video_denoised_combi\bike-packing.mp4


Processing video blackswan: 100%|██████████| 50/50 [01:14<00:00,  1.49s/it]


...

Vidéos sauvegardées :
 - Noised : video_noised_combi\elephant.mp4
 - Denoised : video_denoised_combi\elephant.mp4


Processing video flamingo: 100%|██████████| 80/80 [02:18<00:00,  1.73s/it]


Vidéos sauvegardées :
 - Noised : video_noised_combi\flamingo.mp4
 - Denoised : video_denoised_combi\flamingo.mp4


Processing video goat:  51%|█████     | 46/90 [1:11:48<1:08:40, 93.65s/it]   


KeyboardInterrupt: 